In [ ]:
from dash import Dash, dcc, html, Input, Output, dash_table, State
import pandas as pd
import dash_leaflet as dl
import plotly.express as px
from animalShelter import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "ankur1"
shelter = AnimalShelter(username, password)

def fetch_data(query={}):
    records = shelter.read(query)
    for record in records:
        record['_id'] = str(record['_id'])
    return records

def classify_rescue(record):
    sex = record.get('sex_upon_outcome', '')
    age_weeks = record.get('age_upon_outcome_in_weeks', 0)
    breed = record.get('breed', '')
    
    water_rescue_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
    mountain_rescue_breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
    disaster_rescue_breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    
    if breed in water_rescue_breeds and sex == 'Intact Female' and 26 <= age_weeks <= 156:
        return 'Water Rescue'
    elif breed in mountain_rescue_breeds and sex == 'Intact Male' and 26 <= age_weeks <= 156:
        return 'Mountain or Wilderness Rescue'
    elif breed in disaster_rescue_breeds and sex == 'Intact Male' and 20 <= age_weeks <= 300:
        return 'Disaster or Individual Tracking'
    else:
        return 'Other'

# Initial data loading
df_initial = pd.DataFrame(fetch_data())

app = Dash(__name__)
app.layout = html.Div([
    html.Div([
        html.Img(src='/assets/logo.png', style={'width': 'auto', 'height': '100px'}),
        html.Div('dashboard created by: Ankur Tandan', style={'textAlign': 'left', 'margin': '10px'})
    ]),
    dcc.RadioItems(
        id='type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df_initial.columns],
        data=df_initial.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[]
    ),
    dl.Map(id='map-id', center=[30.2672, -97.7431], zoom=10, children=[
        dl.TileLayer()
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}),
    dcc.Graph(id='pie-chart', style={'width': '100%', 'height': '40vh'})
])

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'selected_rows')],
    [State('datatable-id', 'data')]
)
def update_map(selected_rows, rows):
    if not selected_rows or not rows:
        # If no rows are selected, return the default map layout without markers.
        return [dl.TileLayer()]
    selected_row = rows[selected_rows[0]]
    if 'location_lat' in selected_row and 'location_long' in selected_row and 'name' in selected_row and 'breed' in selected_row:
        return [
            dl.TileLayer(),
            dl.Marker(position=[selected_row['location_lat'], selected_row['location_long']], children=[
                dl.Tooltip(f"{selected_row['name']} - {selected_row['breed']}"),  # Show both name and breed in the tooltip
                dl.Popup([
                    html.H2("Animal Details"),
                    html.P(f"Name: {selected_row['name']}"),
                    html.P(f"Breed: {selected_row['breed']}")
                ])
            ])
        ]

@app.callback(
    [Output('datatable-id', 'data'), Output('pie-chart', 'figure')],
    [Input('type-filter', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'all':
        df_filtered = pd.DataFrame(fetch_data())
    else:
        df_filtered = pd.DataFrame(fetch_data())
        df_filtered = df_filtered[df_filtered.apply(classify_rescue, axis=1) == filter_type]

    data_table = df_filtered.to_dict('records')
    
    # Update the pie chart to show the distribution of breeds
    if not df_filtered.empty and 'breed' in df_filtered.columns:
        pie_fig = px.pie(df_filtered, names='breed', title="Distribution of Breeds")
    else:
        pie_fig = {'layout': {'title': 'No data available'}}

    return data_table, pie_fig

if __name__ == '__main__':
    app.run_server()



